In [1]:
# lecture 2 code
using PyPlot
using BenchmarkTools

In [2]:
#求解上三角矩阵
function back_sub(A, b)
    x = zeros(n)
    for i = n:-1:1
        x[i] = b[i]
        for j = i + 1 : n
            x[i] -= A[i, j] * x[j]
        end
        x[i] = x[i] / A[i, i]
    end
    return x
end

back_sub (generic function with 1 method)

In [3]:
#高斯消元法
function linear_solver(A, b)
    n = length(b)

    for row = 1:n
        # println(row)
        for i = row + 1 : n
            # println(i)
            coef = A[i, row] / A[row, row]
            for j = row : n
                A[i, j] = A[i, j] - coef * A[row, j]
                # println(j)
                
            end
            # 课上的程序缺少了这行
            b[i] = b[i] - coef * b[row]
        end
        # println(A)
    end
    return back_sub(A,b)
end

linear_solver (generic function with 1 method)

In [4]:
#修改高斯消元法(不带排序，把A和b合并处理)
function modify_linear_solver(A, b)
    n = length(b)
    C =hcat(A,b)
    # println(C)
    for row = 1:n
        # println("row = ",row)
        for i = row + 1 : n
            # println("i = ",i)
            coef = C[i, row] / C[row, row]
            for j = row : n+1
                # println("j = ",j)
                C[i, j] = C[i, j] - coef * C[row, j]
            end
        end     
        # println(C)
    end
    D = C[:,1:n]
    e = C[:,n+1]
    # println(D)
    # println(e)
    return back_sub(D,e)
end

modify_linear_solver (generic function with 1 method)

In [5]:
#随机生成n阶矩阵
n = 100
A = zeros(n, n)
for i = 1:n
    for j = 1:n
        A[i, j] = randn(1)[1]
    end
end
b = randn(n);

In [6]:
modify_linear_solver(A, b)
@btime modify_linear_solver(A, b)

  899.300 μs (30307 allocations: 712.84 KiB)


100-element Vector{Float64}:
  -33.466714999619356
   13.467865098240647
  -38.09012447521151
   12.986178269226231
  -42.179135557519956
  -65.01871955946109
  -29.868921635318053
   63.71740114783143
   39.23806830964172
   44.853329361606654
    ⋮
  -22.282280531905204
  -41.830145806365415
  -50.07361365581758
  -52.542721175923084
    4.220248915946266
   35.880074375584385
   31.671115875053523
 -142.8516974324281
   -7.744102573310137

In [7]:
linear_solver(A, b)
@btime linear_solver(A, b)

  2.334 ms (30302 allocations: 554.81 KiB)


100-element Vector{Float64}:
  -33.46671499959226
   13.46786509823526
  -38.09012447520575
   12.986178269273594
  -42.17913555752505
  -65.01871955947537
  -29.868921635351278
   63.71740114782126
   39.23806830963633
   44.85332936164094
    ⋮
  -22.28228053190367
  -41.83014580636185
  -50.07361365581389
  -52.542721175919226
    4.220248915945809
   35.88007437558173
   31.671115875051537
 -142.85169743241767
   -7.744102573309677

In [8]:
A \ b
@btime A\b

  1.290 ms (4 allocations: 79.92 KiB)


100-element Vector{Float64}:
  -33.46671499960197
   13.467865098245422
  -38.09012447521404
   12.986178269245313
  -42.179135557513696
  -65.0187195594542
  -29.868921635333315
   63.71740114781558
   39.23806830965057
   44.85332936161462
    ⋮
  -22.28228053190383
  -41.83014580636137
  -50.07361365581391
  -52.54272117591922
    4.220248915945799
   35.88007437558172
   31.67111587505153
 -142.85169743241767
   -7.744102573309677

In [9]:
#带排序的高斯消元法（列主元）
function sort_gauss_elimination(A, b)
    n = length(b)

    #排序
    for k = 1 : n-1  #从1到n-1列，最后一列不需要排序，因为就一个
        row_max = k #默认第k行k列是最大的
        for m = k+1 :n  #从k+1到n行
            #找最大行的位置
            if abs(A[m,k]) > abs(A[row_max,k])
                row_max = m
            end
        end

        #判断是否需要交换
        if row_max != k #需要交换
            # 使用临时变量temp_row来存储第k行的内容
            temp_row = A[k, :]
            temp_b = b[k]
            
            # 将第max_row行的内容复制到第k行
            A[k, :] = A[row_max, :]
            b[k] = b[row_max]
            
            # 将之前第k行（现在存储在temp变量中）的内容复制到第max_row行
            A[row_max, :] = temp_row
            b[row_max] = temp_b
        end

    end

    #消元，化上三角
    for row = 1:n
        for i = row + 1 : n
            coef = A[i, row] / A[row, row]
            for j = row : n
                A[i, j] = A[i, j] - coef * A[row, j]
            end
            b[i] = b[i] - coef * b[row]
        end
    end
    return back_sub(A,b)
end

sort_gauss_elimination (generic function with 1 method)

In [11]:
@btime sort_gauss_elimination(A, b)

  31.000 μs (1262 allocations: 23.53 KiB)


20-element Vector{Float64}:
 -0.28331760642343606
  0.2922862128947752
 -0.7623893248715793
 -7.229197176079462e-5
 -0.7835164744438554
 -0.23166660255487856
  0.04171623488989506
  0.6942279338391302
  0.17623488774638388
  0.7541059998942519
 -0.24973557466210458
  0.18865074426940903
  0.25068055670816086
 -0.13065851548563379
  1.1396246141760171
 -0.1731625582546033
  0.5042807477277734
  0.9959088080866131
  0.1850481901861238
  0.7536658913229881

In [12]:
@btime A\b

  3.312 μs (3 allocations: 3.69 KiB)


20-element Vector{Float64}:
 -0.2833176064234382
  0.2922862128947741
 -0.7623893248715803
 -7.229197176127236e-5
 -0.7835164744438561
 -0.23166660255487892
  0.04171623488989653
  0.6942279338391307
  0.1762348877463838
  0.7541059998942512
 -0.24973557466210583
  0.1886507442694091
  0.2506805567081612
 -0.13065851548563437
  1.139624614176018
 -0.1731625582546034
  0.5042807477277734
  0.9959088080866131
  0.18504819018612373
  0.7536658913229881

In [10]:
#测试算例,随机生成n阶矩阵
n = 20
A = zeros(n, n)
for i = 1:n
    for j = 1:n
        A[i, j] = randn(1)[1]
    end
end
b = randn(n);